In [ ]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.externals import joblib
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
from numpy.random import seed
import matplotlib.pyplot as plt
import random
import time
from sklearn.utils import shuffle
import copy
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import keras
%matplotlib inline

In [ ]:
data_dir = 'drive/My Drive/rpt_project/rpt_data/4walls_materials_RawData/'
result_dir = 'drive/My Drive/rpt_project/rpt_data/results/'

calibration_points = 'drive/My Drive/rpt_project/rpt_data/old4walls_materials/r_10527_5M.mac'

data_files =  ['r_16_10527_5M_Air_Air_counts_RawData.txt',
               'r_16_10527_5M_AC_Air_counts_RawData.txt',
               'r_16_10527_5M_AC_Water_counts_RawData.txt',
               'r_16_10527_5M_AC_Glass_counts_RawData.txt',
               'r_16_10527_5M_AC_Steel_counts_RawData.txt',
               'r_16_10527_5M_AL_Air_counts_RawData.txt',
               'r_16_10527_5M_AL_Water_counts_RawData.txt',
               'r_16_10527_5M_AL_Glass_counts_RawData.txt',
               'r_16_10527_5M_AL_Steel_counts_RawData.txt',
               'r_16_10527_5M_SS_Air_counts_RawData.txt',
               'r_16_10527_5M_SS_Water_counts_RawData.txt',
               'r_16_10527_5M_SS_Glass_counts_RawData.txt']

num_tasks = len(data_files)

In [ ]:
# Performance
def evaluate_performance(real, pred):
    diff_sq = (real - pred) ** 2
    mede = np.sum(np.sqrt(np.sum(diff_sq, axis=1))) / pred.shape[0]

    diff_abs = np.abs(real - pred)
    mae = np.sum(diff_abs, axis=0) / pred.shape[0]

    standard_deviation = np.std(diff_abs, axis=0)

    r2 = r2_score(real, pred)
    rmse = sqrt(mean_squared_error(real, pred))

    return mede*1000, r2, rmse*1000

In [ ]:
def read_position_from_mac(calibration_file):
    """Read the positions of tracers from .mac file

    Args:
        calibration_file (str): Name of the file containing Geant4 macro.
    """
    simulated_positions = []
    for line in open(calibration_file):
        temp = line.split()
        if "/gun/position" in line:
            simulated_positions.append([float(temp[1]), float(temp[2]), float(temp[3])])
    simulated_positions = np.array(simulated_positions, dtype=float)
    return simulated_positions

In [ ]:
def create_ann_model():

    layers = [256, 128, 128, 16, 3]

    NN_model = Sequential()

    # The Input Layer :
    NN_model.add(Dense(layers[0], kernel_initializer='normal', input_dim=18))
    # NN_model.add(Dropout(0.2))
    # NN_model.add(BatchNormalization())
    # NN_model.add(Activation(activation))

    # The Hidden Layers :
    for i in range(1, len(layers)-1):
        NN_model.add(Dense(layers[i], kernel_initializer='normal'))
        # NN_model.add(Dropout(0.2))
        # NN_model.add(BatchNormalization())
        NN_model.add(Activation('relu'))

    # The Output Layer :
    NN_model.add(Dense(layers[-1], kernel_initializer='normal', activation='linear'))


    # Compile the network :
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    NN_model.compile(loss='mean_squared_error', optimizer=opt)

    return NN_model

In [ ]:
sample = 1000
num_features = 16
seed(2020)

In [ ]:
with open(result_dir + 'train_data_8527.npy', 'rb') as f:
    X_train_all = np.load(f)

# with open(result_dir + 'val_data.npy', 'rb') as f:
#     X_val_all = np.load(f)

with open(result_dir + 'test_data_2000.npy', 'rb') as f:
    X_test_all = np.load(f)

with open(result_dir + 'train_label_8527.npy', 'rb') as f:
    y_train_all = np.load(f)

# with open(result_dir + 'val_label.npy', 'rb') as f:
#     y_val_all = np.load(f)

with open(result_dir + 'test_label_2000.npy', 'rb') as f:
    y_test_all = np.load(f)

In [ ]:
material_combinations = ['Air-Air', 'AC-Air', 'AC-Water', 'AC-Glass', 'AC-Steel', 'AL-Air',
                         'AL-Water', 'AL-Glass', 'AL-Steel', 'SS-Air', 'SS-Water', 'SS-Glass']
num_combs = len(set(material_combinations)) 
Zeff = {'Water': 7.42, 'Air': 8.02, 'AC': 5.57, 'Glass': 11.5, 'AL': 9.31, 'SS': 24.6, 'Steel': 24.6}
density = {'Water': 1, 'Air': 0.0012, 'AC': 1.19, 'Glass': 2.4, 'AL': 3.97, 'SS': 8.02, 'Steel': 8.02}

In [ ]:
# SVR PT and BL experiments
mede_arr_pt, r2_arr_pt, rmse_arr_pt = [], [], []

for i in range(num_tasks):

    wall = material_combinations[i].split('-')[0]
    media = material_combinations[i].split('-')[1]
    # new_features = [[Zeff[wall], Zeff[media], density[wall], density[media]]] * sample
    new_features = [[Zeff[media], density[media]]] * sample
    
    X_cand, y_cand = X_train_all[8527*i:8527*(i+1)], y_train_all[8527*i:8527*(i+1)]

    X_cand, y_cand = shuffle(X_cand, y_cand)
    X_train, y_train = X_cand[:sample], y_cand[:sample]
    X_train = np.hstack((X_train, new_features))

    m, r, s = [], [], []

    reg_x = SVR(C=64, gamma=1024, epsilon=0.0001220703125, kernel='rbf', tol=1e-4)
    reg_y = SVR(C=64, gamma=1024, epsilon=0.0001220703125, kernel='rbf', tol=1e-4)
    reg_z = SVR(C=64, gamma=1024, epsilon=0.0001220703125, kernel='rbf', tol=1e-4)

    # reg_x = SVR(C=1024, gamma=128, epsilon=6.103515625e-05, kernel='rbf', tol=1e-4)
    # reg_y = SVR(C=1024, gamma=128, epsilon=6.103515625e-05, kernel='rbf', tol=1e-4)
    # reg_z = SVR(C=1024, gamma=128, epsilon=6.103515625e-05, kernel='rbf', tol=1e-4)

    # reg_x = SVR(C=2048, gamma=512, epsilon=3.0517578125e-05, kernel='rbf', tol=1e-4)
    # reg_y = SVR(C=2048, gamma=512, epsilon=3.0517578125e-05, kernel='rbf', tol=1e-4)
    # reg_z = SVR(C=2048, gamma=512, epsilon=3.0517578125e-05, kernel='rbf', tol=1e-4)

    reg_x.fit(X_train, y_train[:, 0])
    reg_y.fit(X_train, y_train[:, 1])
    reg_z.fit(X_train, y_train[:, 2])

    for j in range(num_tasks):
        wall = material_combinations[j].split('-')[0]
        media = material_combinations[j].split('-')[1]
        # new_features = [[Zeff[wall], Zeff[media], density[wall], density[media]]] * 2000
        new_features = [[Zeff[media], density[media]]] * 2000

        X_test, y_test = X_test_all[2000*j:2000*(j+1)], y_test_all[2000*j:2000*(j+1)]
        X_test = np.hstack((X_test, new_features))

        recons_x = reg_x.predict(X_test)
        recons_y = reg_y.predict(X_test)
        recons_z = reg_z.predict(X_test)

        recons = np.stack((recons_x, recons_y, recons_z), axis=-1)
        
        mede, r2, rmse = evaluate_performance(y_test, recons)
        m.append(mede)
        r.append(r2)
        s.append(rmse)

    mede_arr_pt.append(m)
    r2_arr_pt.append(r)
    rmse_arr_pt.append(s)

In [ ]:
# ANN BL and PT experiments
mede_arr, r2_arr, rmse_arr = [], [], []

for i in range(num_tasks):

    wall = material_combinations[i].split('-')[0]
    media = material_combinations[i].split('-')[1]
    # new_features = [[Zeff[wall], Zeff[media], density[wall], density[media]]] * sample
    new_features = [[Zeff[media], density[media]]] * 8527
    new_features = np.exp(new_features)

    X_cand, y_cand = X_train_all[8527*i:8527*(i+1)], y_train_all[8527*i:8527*(i+1)]

    X_cand, y_cand = shuffle(X_cand, y_cand)
    X_cand = np.hstack((X_cand, new_features))
    X_train, y_train = X_cand[:sample], y_cand[:sample]
   


    # scaling
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_cand = scaler.transform(X_cand)

    X_train, y_train = shuffle(X_train, y_train)
    X_train, y_train = X_train[:sample], y_train[:sample]

    # X_val, y_val = X_val_all[2527*i:2527*(i+1)], y_val_all[2527*i:2527*(i+1)]
    # X_val = scaler.transform(X_val)

    # X_train = np.concatenate((X_train, X_val))
    # y_train = np.concatenate((y_train, y_val))

    opt = keras.optimizers.Adam(learning_rate=0.0001)
    NN_model = create_ann_model()

    my_callbacks = [
    keras.callbacks.EarlyStopping(patience=3),]
    # keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    # keras.callbacks.TensorBoard(log_dir='./logs')]

    NN_model.fit(X_train, y_train, 
                 validation_data=(X_cand, y_cand), 
                 epochs=100, 
                 batch_size=32, 
                 callbacks=my_callbacks, 
                 verbose=0)

    m, r, s = [], [], []

    for j in range(num_tasks):

        wall = material_combinations[j].split('-')[0]
        media = material_combinations[j].split('-')[1]
        # new_features = [[Zeff[wall], Zeff[media], density[wall], density[media]]] * 2000
        new_features = [[Zeff[media], density[media]]] * 2000
        new_features = np.exp(new_features)

        X_test, y_test = X_test_all[2000*j:2000*(j+1)], y_test_all[2000*j:2000*(j+1)]
        X_test = np.hstack((X_test, new_features))
        X_test = scaler.transform(X_test)

        recons = NN_model.predict(X_test, verbose=0)

        mede, r2, rmse = evaluate_performance(y_test, recons)
        m.append(mede)
        r.append(r2)
        s.append(rmse)
        
    mede_arr.append(m)
    r2_arr.append(r)
    rmse_arr.append(s)

In [ ]:
df_mede = pd.DataFrame(mede_arr_pt)

In [ ]:
df_mede

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.521964,9.900859,89.771385,89.650033,90.225042,9.739015,90.440729,90.235904,89.453349,10.897765,89.602661,90.142798
1,22.387669,0.601992,86.949444,87.202906,87.690267,9.299193,88.432275,87.462396,86.923261,15.772153,87.086705,87.499656
2,88.804586,88.177717,0.506519,88.747504,89.175894,89.593881,3.868572,88.800869,88.293844,89.362249,3.493329,88.869446
3,88.091800,88.260049,88.040470,0.544861,88.634029,89.044659,89.007670,1.506877,87.828821,88.605259,87.986653,6.770469
4,88.190931,88.349879,88.102215,88.235675,0.795423,89.248259,89.320686,88.748749,1.861412,88.751421,88.181726,88.656102
5,14.746020,7.280247,86.107280,86.587613,87.242643,0.635446,87.784772,86.872833,85.942527,4.737229,86.165727,86.657321
6,86.708418,86.757627,3.194675,87.020663,87.632480,87.862636,0.486342,87.288710,86.415214,87.591994,4.411395,87.119485
7,87.591185,88.223341,87.614312,1.827871,89.629205,89.240953,90.369312,0.502938,87.457425,89.392160,87.856570,7.852353
8,87.762988,87.413751,87.414057,87.510854,2.185578,88.508824,88.590742,87.792231,0.773444,88.114404,87.430043,87.862500
9,12.509480,9.123053,90.267103,91.751550,92.422061,3.534990,93.206171,91.290755,90.083419,0.637794,90.344094,91.004634


In [ ]:
# SVR PT and BL experiments
mede_arr_pt, r2_arr_pt, rmse_arr_pt = [], [], []

for i in range(num_tasks):

    X_cand, y_cand = X_train_all[8527*i:8527*(i+1)], y_train_all[8527*i:8527*(i+1)]

    X_cand, y_cand = shuffle(X_cand, y_cand)
    X_train, y_train = X_cand[:sample], y_cand[:sample]
    X_train = np.log(X_train)

    m, r, s = [], [], []

    reg_x = SVR(C=64, gamma=1024, epsilon=0.0001220703125, kernel='rbf', tol=1e-4)
    reg_y = SVR(C=64, gamma=1024, epsilon=0.0001220703125, kernel='rbf', tol=1e-4)
    reg_z = SVR(C=64, gamma=1024, epsilon=0.0001220703125, kernel='rbf', tol=1e-4)

    # reg_x = SVR(C=1024, gamma=128, epsilon=6.103515625e-05, kernel='rbf', tol=1e-4)
    # reg_y = SVR(C=1024, gamma=128, epsilon=6.103515625e-05, kernel='rbf', tol=1e-4)
    # reg_z = SVR(C=1024, gamma=128, epsilon=6.103515625e-05, kernel='rbf', tol=1e-4)

    # reg_x = SVR(C=2048, gamma=512, epsilon=3.0517578125e-05, kernel='rbf', tol=1e-4)
    # reg_y = SVR(C=2048, gamma=512, epsilon=3.0517578125e-05, kernel='rbf', tol=1e-4)
    # reg_z = SVR(C=2048, gamma=512, epsilon=3.0517578125e-05, kernel='rbf', tol=1e-4)

    reg_x.fit(X_train, y_train[:, 0])
    reg_y.fit(X_train, y_train[:, 1])
    reg_z.fit(X_train, y_train[:, 2])

    for j in range(num_tasks):

        X_test, y_test = X_test_all[2000*j:2000*(j+1)], y_test_all[2000*j:2000*(j+1)]
        X_test = np.log(X_test)

        recons_x = reg_x.predict(X_test)
        recons_y = reg_y.predict(X_test)
        recons_z = reg_z.predict(X_test)

        recons = np.stack((recons_x, recons_y, recons_z), axis=-1)
        
        mede, r2, rmse = evaluate_performance(y_test, recons)
        m.append(mede)
        r.append(r2)
        s.append(rmse)

    mede_arr_pt.append(m)
    r2_arr_pt.append(r)
    rmse_arr_pt.append(s)

In [ ]:
df_mede = pd.DataFrame(mede_arr_pt)

In [ ]:
df_mede

,0,1,2,3,4,5,6,7,8,9,10,11
0,86.066123,86.362157,77.435867,86.571342,87.250434,87.302751,87.824295,86.825811,85.820258,87.141734,86.071513,86.563817
1,86.079797,86.299229,85.951391,86.533569,87.193449,87.276266,87.794076,86.763542,85.810621,87.116649,86.050679,86.532495
2,77.564961,86.295879,86.003855,86.532037,87.156762,87.324673,87.795907,86.762920,85.892198,87.125271,86.118008,86.586556
3,86.128745,86.287624,85.974112,86.572930,87.221815,87.295126,87.847481,86.753409,85.837501,87.169601,86.063453,86.538755
4,86.115872,86.265353,85.944460,86.594218,87.241785,87.308707,87.906359,86.758024,85.824094,87.186069,86.069444,86.538498
5,86.192336,86.301328,86.018089,86.556195,87.186441,87.314895,87.808984,86.755458,85.887334,87.157174,86.100079,86.571423
6,86.053613,86.291132,85.929724,86.574473,87.244926,87.278392,87.861798,86.768466,85.787603,87.157433,86.013138,86.517443
7,86.185970,86.264613,85.984460,86.588827,87.215393,87.328795,87.891552,86.750814,85.875000,87.189903,86.103101,86.564854
8,86.115289,86.304214,85.971168,86.525783,87.170735,87.292994,87.782433,86.767769,85.839722,87.110055,86.076981,86.554873
9,86.055457,86.302462,85.936435,86.553720,87.222671,87.280100,87.828959,86.777068,85.795474,87.131484,86.046170,86.528863
